In [13]:

### File Address: https://www.sample-videos.com/
%load_ext autoreload
%autoreload 2
import json
video_path = "demo-3s-small.mp4"
input_width = 240
input_height = 320
output_width = 120
output_height = 180
snark_path = "/Users/jojo/Documents/__research/v5/fibonacci/video-resizing-freivalds"


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Prover

### Decompose images from the video

In [14]:
# Use ffmpeg to decompose
import os
# Create directories if they don't exist
if not os.path.exists("frames"):
    os.makedirs("frames")
if not os.path.exists("decomposed_frames"):
    os.makedirs("decomposed_frames")

os.system(f"ffmpeg -i {video_path} frames/output_%04d.png")

ffmpeg version 7.1 Copyright (c) 2000-2024 the FFmpeg developers
  built with Apple clang version 16.0.0 (clang-1600.0.26.4)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/7.1_4 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags='-Wl,-ld_classic' --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libharfbuzz --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --e

0

In [15]:
import sys
sys.path.append(os.path.abspath("../"))
from scripts.decompose_image import decompose_image 
for i in sorted(os.listdir("frames")):
    if i.endswith(".png"):
        r,g,b = decompose_image(os.path.join("frames", i), os.path.join("decomposed_frames", i[:-4]))

Saved R channel to decomposed_frames/output_0001_R.txt
Saved G channel to decomposed_frames/output_0001_G.txt
Saved B channel to decomposed_frames/output_0001_B.txt
Saved R channel to decomposed_frames/output_0002_R.txt
Saved G channel to decomposed_frames/output_0002_G.txt
Saved B channel to decomposed_frames/output_0002_B.txt
Saved R channel to decomposed_frames/output_0003_R.txt
Saved G channel to decomposed_frames/output_0003_G.txt
Saved B channel to decomposed_frames/output_0003_B.txt
Saved R channel to decomposed_frames/output_0004_R.txt
Saved G channel to decomposed_frames/output_0004_G.txt
Saved B channel to decomposed_frames/output_0004_B.txt
Saved R channel to decomposed_frames/output_0005_R.txt
Saved G channel to decomposed_frames/output_0005_G.txt
Saved B channel to decomposed_frames/output_0005_B.txt
Saved R channel to decomposed_frames/output_0006_R.txt
Saved G channel to decomposed_frames/output_0006_G.txt
Saved B channel to decomposed_frames/output_0006_B.txt
Saved R ch

In [16]:
### Generate the targeted File 
## Shared knowledge between prover and verifier
if not os.path.exists("resized.mp4"):
    os.system(f"ffmpeg -i {video_path} -vf scale=180:120 resized.mp4")
if not os.path.exists("resized_frames"):
    os.makedirs("resized_frames")
if not os.path.exists("decomposed_resized_frames"):
    os.makedirs("decomposed_resized_frames")

# Check if resized.mp4 exists and ask for overwrite permission

os.system(f"ffmpeg -i resized.mp4 resized_frames/output_%04d.png")
for i in sorted(os.listdir("frames")):
    if i.endswith(".png"):
        r,g,b = decompose_image(os.path.join("resized_frames", i), os.path.join("decomposed_resized_frames", i[:-4]))

ffmpeg version 7.1 Copyright (c) 2000-2024 the FFmpeg developers
  built with Apple clang version 16.0.0 (clang-1600.0.26.4)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/7.1_4 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags='-Wl,-ld_classic' --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libharfbuzz --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --e

Saved R channel to decomposed_resized_frames/output_0001_R.txt
Saved G channel to decomposed_resized_frames/output_0001_G.txt
Saved B channel to decomposed_resized_frames/output_0001_B.txt
Saved R channel to decomposed_resized_frames/output_0002_R.txt
Saved G channel to decomposed_resized_frames/output_0002_G.txt
Saved B channel to decomposed_resized_frames/output_0002_B.txt
Saved R channel to decomposed_resized_frames/output_0003_R.txt
Saved G channel to decomposed_resized_frames/output_0003_G.txt
Saved B channel to decomposed_resized_frames/output_0003_B.txt
Saved R channel to decomposed_resized_frames/output_0004_R.txt
Saved G channel to decomposed_resized_frames/output_0004_G.txt
Saved B channel to decomposed_resized_frames/output_0004_B.txt
Saved R channel to decomposed_resized_frames/output_0005_R.txt
Saved G channel to decomposed_resized_frames/output_0005_G.txt
Saved B channel to decomposed_resized_frames/output_0005_B.txt
Saved R channel to decomposed_resized_frames/output_000

In [17]:
# Generate Proof from rust 
if not os.path.exists("proof"):
    os.makedirs("proof")
os.system(f"cd {snark_path}/program && cargo prove build")
current_dir = os.getcwd()
base_dir = os.path.dirname(current_dir)
for i in sorted(os.listdir("decomposed_frames")):
    print("generating proof for ", i)
    if i.endswith(".txt"):
        config = {
            "input_width": input_width,
            "input_height": input_height,
            "output_width": output_width,
            "output_height": output_height,
            "input_file": os.path.join(base_dir, "demo/decomposed_frames", i),
            "target_file": os.path.join(base_dir, "demo/decomposed_resized_frames", i),
            "target_prove_file": os.path.join(base_dir, "demo/proof", i[:-4] + "_proof-with-pis.bin"),
            "target_vk_file": os.path.join(base_dir, "demo/proof", i[:-4] + "_vk.bin")
        }
        with open(os.path.join(snark_path, "script", "config.json"), "w") as f:
            json.dump(config, f)
        os.system(f"cd {snark_path}/script && RUST_LOG=info cargo run --release -- --prove")
    break
        


cargo:warning=rustc +succinct --version: "rustc 1.82.0-dev\n"


[sp1]      Finished `release` profile [optimized] target(s) in 0.17s


cargo:rustc-env=SP1_ELF_fibonacci-program=/Users/jojo/Documents/__research/v5/fibonacci/video-resizing-freivalds/target/elf-compilation/riscv32im-succinct-zkvm-elf/release/fibonacci-program
generating proof for  output_0001_B.txt


   Compiling script v0.1.0 (/Users/jojo/Documents/__research/v5/fibonacci/video-resizing-freivalds/script)
 --> script/src/bin/main.rs:1:11
  |
1 | use lib::{generate_horizontal_filter, generate_vertical_filter, load_image_from_file};
  |           ^^^^^^^^^^^^^^^^^^^^^^^^^^  ^^^^^^^^^^^^^^^^^^^^^^^^
  |
  = note: `#[warn(unused_imports)]` on by default

 --> script/src/bin/main.rs:2:72
  |
2 | use sp1_sdk::{utils, ProverClient, SP1ProofWithPublicValues, SP1Stdin, SP1VerifyingKey};
  |                                                                        ^^^^^^^^^^^^^^^

 --> script/src/bin/main.rs:5:5
  |
5 | use clap::Parser;
  |     ^^^^^^^^^^^^

   --> script/src/bin/main.rs:157:32
    |
157 |     let client = ProverClient::new();
    |                                ^^^
    |
    = note: `#[warn(deprecated)]` on by default

  --> script/src/bin/main.rs:23:9
   |
23 |     let INPUT_WIDTH= config["input_width"].as_i64().expect("Missing input_width") as i32;
   |         ^^^^^^^^^^^

input_path: "decomposed_frames/output_0001_B.txt"


thread 'main' panicked at lib/src/lib.rs:13:40:
Failed to open input file: Os { code: 2, kind: NotFound, message: "No such file or directory" }
note: run with `RUST_BACKTRACE=1` environment variable to display a backtrace


# Verifier

In [18]:
if not os.path.exists("resized_frames"):
    os.makedirs("resized_frames")
if not os.path.exists("decomposed_resized_frames"):
    os.makedirs("decomposed_resized_frames")

# Check if resized.mp4 exists and ask for overwrite permission

os.system(f"ffmpeg -i resized.mp4 resized_frames/output_%04d.png")
for i in sorted(os.listdir("frames")):
    if i.endswith(".png"):
        r,g,b = decompose_image(os.path.join("resized_frames", i), os.path.join("decomposed_resized_frames", i[:-4]))

Saved R channel to decomposed_resized_frames/output_0001_R.txt
Saved G channel to decomposed_resized_frames/output_0001_G.txt
Saved B channel to decomposed_resized_frames/output_0001_B.txt
Saved R channel to decomposed_resized_frames/output_0002_R.txt
Saved G channel to decomposed_resized_frames/output_0002_G.txt
Saved B channel to decomposed_resized_frames/output_0002_B.txt
Saved R channel to decomposed_resized_frames/output_0003_R.txt
Saved G channel to decomposed_resized_frames/output_0003_G.txt
Saved B channel to decomposed_resized_frames/output_0003_B.txt
Saved R channel to decomposed_resized_frames/output_0004_R.txt
Saved G channel to decomposed_resized_frames/output_0004_G.txt
Saved B channel to decomposed_resized_frames/output_0004_B.txt
Saved R channel to decomposed_resized_frames/output_0005_R.txt
Saved G channel to decomposed_resized_frames/output_0005_G.txt
Saved B channel to decomposed_resized_frames/output_0005_B.txt
Saved R channel to decomposed_resized_frames/output_000

ffmpeg version 7.1 Copyright (c) 2000-2024 the FFmpeg developers
  built with Apple clang version 16.0.0 (clang-1600.0.26.4)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/7.1_4 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags='-Wl,-ld_classic' --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libharfbuzz --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --e

Saved G channel to decomposed_resized_frames/output_0021_G.txt
Saved B channel to decomposed_resized_frames/output_0021_B.txt
Saved R channel to decomposed_resized_frames/output_0022_R.txt
Saved G channel to decomposed_resized_frames/output_0022_G.txt
Saved B channel to decomposed_resized_frames/output_0022_B.txt
Saved R channel to decomposed_resized_frames/output_0023_R.txt
Saved G channel to decomposed_resized_frames/output_0023_G.txt
Saved B channel to decomposed_resized_frames/output_0023_B.txt
Saved R channel to decomposed_resized_frames/output_0024_R.txt
Saved G channel to decomposed_resized_frames/output_0024_G.txt
Saved B channel to decomposed_resized_frames/output_0024_B.txt
Saved R channel to decomposed_resized_frames/output_0025_R.txt
Saved G channel to decomposed_resized_frames/output_0025_G.txt
Saved B channel to decomposed_resized_frames/output_0025_B.txt
Saved R channel to decomposed_resized_frames/output_0026_R.txt
Saved G channel to decomposed_resized_frames/output_002